In [11]:
import os,random
os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ["THEANO_FLAGS"]  = "device=gpu%d"%(1)
os.environ["MKL_THREADING_LAYER"] = "GNU"
import numpy as np
from keras.utils import np_utils
import keras.models as models
from keras.layers.core import Reshape,Dense,Dropout,Activation,Flatten
from keras.layers.noise import GaussianNoise
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.regularizers import *
from keras.optimizers import adam
import random, sys, keras

In [21]:
from util import dataset_load

datafile_src = 'radio_data_2018Aug03_04_06.data'
#modelfile = 'QPSK.wts_model.h5'
[X_train_src, Y_train_src, X_test_src, Y_test_src] = dataset_load(datafile_src)

datafile_tar = 'radio_data_2018Aug03_04_10.data'
[X_train_tar, Y_train_tar, X_test_tar, Y_test_tar] = dataset_load(datafile_tar)

print('done')

done


In [ ]:
import da_tool.tca
my_tca = da_tool.tca.TCA(dim=3,kerneltype='rbf', kernelparam=1, mu=1)
#x_src_tca = np.zeros([10000,2,4])
#x_tar_tca = np.zeros([10000,2,4])
#x_tar_o_tca = np.zeros([10000,2,4])
#x_src_tca_test = np.zeros([10000,2,4])
#x_tar_tca_test = np.zeros([10000,2,4])
#x_tar_o_tca_test = np.zeros([10000,2,4])
#for i in range(0,len(X_train_src)):
X_train_src1 = np.reshape(X_train_src, [10000256])
X_train_tar1 = np.reshape(X_train_tar, [10000, 256])

x_src_tca, x_tar_tca, x_tar_o_tca = my_tca.fit_transform(X_train_src1, X_train_tar1)
#    x_src_tca_test[i], x_tar_tca_test[i], x_tar_o_tca_test[i] = my_tca.fit_transform(X_test_src[i], X_test_tar[i])
print(x_src_tca.shape)
print(len(X_train_src))

/usr/local/lib/python3.5/dist-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full((10000, 10000), 1) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)


In [20]:
print(X_train_src.shape)

print(X_train_src.shape)

(2560000,)
(2560000,)


In [ ]:
import da_tool.tca
my_tca = da_tool.tca.TCA(dim=30,kerneltype='rbf', kernelparam=1, mu=1)
x_src_tca = np.zeros([10000,2,4])
x_tar_tca = np.zeros([10000,2,4])
x_tar_o_tca = np.zeros([10000,2,4])
x_src_tca_test = np.zeros([10000,2,4])
x_tar_tca_test = np.zeros([10000,2,4])
x_tar_o_tca_test = np.zeros([10000,2,4])
for i in range(0,len(X_train_src)):
    x_src_tca[i], x_tar_tca[i], x_tar_o_tca[i] = my_tca.fit_transform(X_train_src[i], X_train_tar[i])
    x_src_tca_test[i], x_tar_tca_test[i], x_tar_o_tca_test[i] = my_tca.fit_transform(X_test_src[i], X_test_tar[i])
print(x_src_tca.shape)
print(x_tar_tca_test.shape)
print(len(X_train_src))

In [ ]:
def deepsensing_network(in_shp = [2, 4], classes = ['busy' ,'idle']):
    K.set_image_dim_ordering('th')
    dr = 0.5
    model = models.Sequential()
    model.add(Reshape([1]+in_shp, input_shape=in_shp))
    model.add(ZeroPadding2D((0, 2)))
    model.add(Convolution2D(256, 1, 3, border_mode='valid', activation="relu", name="conv1", init='glorot_uniform'))
    model.add(Dropout(dr))
    model.add(ZeroPadding2D((0, 2)))
    model.add(Convolution2D(80, 2, 3, border_mode="valid", activation="relu", name="conv2", init='glorot_uniform'))
    model.add(Dropout(dr))
    model.add(Flatten())
    model.add(Dense(256, activation='relu', init='he_normal', name="dense1"))
    model.add(Dropout(dr))
    model.add(Dense( len(classes), init='he_normal', name="dense2" ))
    model.add(Activation('softmax'))
    model.add(Reshape([len(classes)]))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

In [ ]:
from util import dataset_load


in_shp = [2,4]
classes = ['busy', 'idle']
batch_size = 1000
nb_epoch = 100

model = deepsensing_network(in_shp, classes)
     
history = model.fit(x_src_tca,
                    Y_train_src,
                    batch_size=batch_size,
                    nb_epoch=nb_epoch,
                    verbose=2,
                    validation_data=(x_src_tca_test, Y_test_src),
        callbacks = [
            keras.callbacks.ModelCheckpoint(modelfile, monitor='val_loss', verbose=0, save_best_only=True, mode='auto'),
            keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')
        ])

model.load_weights(modelfile)

In [ ]:
test_Y_hat = model.predict(x_tar_tca_test, batch_size=1000)

pd, pf = cal_pd_pf(x_tar_tca_test, Y_test_tar, test_Y_hat)
print(pd, pf)

pds, pfs = cal_roc(x_tar_tca_test, Y_test_tar, test_Y_hat)

plt.plot( pfs, pds)
plt.xlabel('pfs')
plt.ylabel('pds')
plt.show()

In [ ]:
print(pfs)
print(pds)

In [ ]:
from util import dataset_load

datafile = 'radio_data_2018Aug03_03_46.data'
modelfile = 'QPSK.wts_model.h5'

[X_train, Y_train, X_test, Y_test] = dataset_load(datafile)

in_shp = [2,128]
classes = ['busy', 'idle']
batch_size = 1000
nb_epoch = 100

model = deepsensing_network(in_shp, classes)
     
history = model.fit(X_train,
                    Y_train,
                    batch_size=batch_size,
                    nb_epoch=nb_epoch,
                    verbose=2,
                    validation_data=(X_test, Y_test),
        callbacks = [
            keras.callbacks.ModelCheckpoint(modelfile, monitor='val_loss', verbose=0, save_best_only=True, mode='auto'),
            keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')
        ])

model.load_weights(modelfile)

In [ ]:
from util import  cal_roc, plot_roc, cal_pd_pf
import matplotlib.pyplot as plt

datafile = 'radio_data_2018Aug03_04_10.data'

[X_train, Y_train, X_test, Y_test] = dataset_load(datafile)

test_Y_hat = model.predict(X_test, batch_size=1000)

pd, pf = cal_pd_pf(X_test, Y_test, test_Y_hat)
print(pd, pf)

pds, pfs = cal_roc(X_test, Y_test, test_Y_hat)

plt.plot( pfs, pds)
plt.xlabel('pfs')
plt.ylabel('pds')
plt.show()

In [ ]:
import da_tool.tca
my_tca = da_tool.tca.TCA(dim=30,kerneltype='rbf', kernelparam=1, mu=1)
x_src_tca, x_tar_tca, x_tar_o_tca = my_tca.fit_transform(X_test, X_test_new)